# TCC

In [1]:
import pandas as pd
import os as os
import PyPDF2

## Mapeando questões das provas

In [17]:
enem_questions = pd.DataFrame(columns=['id','year','area','body','itens','result','answer'])


path = "../TCC/pdfs"
anos = os.listdir(f'{path}')
for ano in anos:
    
    provas = os.listdir(f'{path}/{ano}')
    for prova in provas:
        with open(f'{path}/{ano}/{prova}' , 'rb') as arquivo_pdf:

        # Criar objeto leitor do PyPDF2
            leitor_pdf = PyPDF2.PdfReader(arquivo_pdf)
            identificador = 1

        # if( "_D2_" in nomeArquivo):
        #     identificador = 91

        # # Loop por todas as páginas do PDF
            for num_pagina in range(len(leitor_pdf.pages)):
            
                pagina = leitor_pdf.pages[num_pagina]
            # print(pagina.extract_text())
                if ("\nQuESTÃO" in pagina.extract_text()) or ("\nQuestão" in pagina.extract_text()):
                    print(pagina.extract_text())
                    
        #         if identificador <= 45:
        #             questoesLinguagens[identificador] = pagina.extract_text().split("QUESTÃO %2d ", identificador)[0]
        #             identificador += 1
        #         elif identificador > 45 and identificador <= 90:
        #             questoesHumanas[identificador] = pagina.extract_text().split("QUESTÃO %2d ", identificador)[0]
        #             identificador += 1
        #         elif identificador > 90 and identificador <=135:
        #             questoesNatureza[identificador] = pagina.extract_text().split("QUESTÃO %2d ", identificador)[0]
        #             identificador += 1
        #         elif identificador > 135:
        #             questoesMatematica[identificador] = pagina.extract_text().split("QUESTÃO %2d - Matemática e suas Tecnologias", identificador)[0]
        #             identificador += 1

        #     else:
        #         if identificador == 46:
        #             redacao.append(pagina.extract_text())
        #         elif (identificador-1  in questoesLinguagens) and identificador <= 45:
        #             questoesLinguagens[identificador-1] += pagina.extract_text()
        #         elif (identificador-1 in questoesHumanas) and identificador > 45 and identificador <= 90:
        #             questoesHumanas[identificador-1]  += pagina.extract_text()              
        #         elif (identificador-1  in questoesNatureza) and identificador <= 135 and identificador > 90:
        #             questoesNatureza [identificador-1] += pagina.extract_text()
        #         elif (identificador-1 in questoesMatematica) and identificador > 135 and identificador <= 180:
        #             questoesMatematica[identificador-1]  += pagina.extract_text()     


*LECZ25dom2*
LC - 2º dia | Caderno 6 - CINZA - Página 2
O projeto Mappi ness, idealizado pela London School of 
Economics , ocupa-se do tema relacionado 
A ao nível de felicidade das pessoas em tempos de 
guerra.
B à dificuldade de medir o nível de felicidade das pessoas a partir de seu humor.
C ao nível de felicidade das pessoas enquanto falam ao celular com seus familiares.
D à relação entre o nível de felicidade das pessoas e o ambiente no qual se encontram.
E à influência das imagens grafitadas pelas ruas no aumento do nível de f elicidade das pessoas.
QuESTÃO 93  
Considere uma charge em que uma aluna, 
em frente a um quadro negro onde se lê a frase 
“REPORTS DUE TODAY!”, diz à sua professora:
Na fase escolar, é prática comum que os professores 
passem atividades extraclasse e marquem uma data para que as mesmas sejam entregues para correção. No caso da cena da charge, a professora ouve uma estudante apresentando argumentos p ara  
A discutir sobre o conteúdo do seu trabalho já en

In [ ]:
def map_questions():
    print()